In [1]:
import time

In [2]:
# pour un ordi avec GPU :

import GPUtil
import torch

In [3]:
torch.cuda.is_available()

True

In [5]:
!set CUDA_PATH=C:\Users\ehess\anaconda3\Lib\site-packages\numba\cuda

import spacy

In [6]:
# spacy.require_gpu()
# spacy.prefer_gpu()
#  j'utilise thinc.api

In [6]:
# pour un ordi avec GPU
# j'utilise thinc

# is_using_gpu = spacy.prefer_gpu()
# if is_using_gpu:
#     print("Using GPU!")
#     torch.set_default_tensor_type("torch.cuda.FloatTensor")
#     print("GPU Usage")
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  1% |  8% |


In [7]:
import pandas as pd
import numpy as np


In [8]:
df = pd.read_csv("contrib_from_csv.csv", sep = ",", encoding = "utf-8", dtype= str)

# sample d'un petit échantillon
df = df.sample(2000)

In [9]:
# supprimer les fin de lignes (deux types de codages)
join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 

# supprimer les UNK des ce corpus (trois types de codages)
kill_texte_illisible = lambda x : x.replace("texte illisible", "")
kill_illisible = lambda x : x.replace("illisible", "")
kill_illisible_plural = lambda x : x.replace("illisibles", "")
kill_illisible_capslock = lambda x : x.replace("ILLISIBLE", "")
kill_illisibleS_capslock = lambda x : x.replace("ILLISIBLES", "")

In [10]:
df["clean"] = df["Contribution"].apply(join_breaklines)
df["clean"] = df["clean"].apply(join_breaklinesn)
df["clean"] = df["clean"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)
df["clean"] = df["clean"].apply(kill_illisible_capslock)
df["clean"] = df["clean"].apply(kill_illisibleS_capslock)

In [12]:
from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu, use_pytorch_for_gpu_memory, set_active_gpu
set_gpu_allocator("pytorch")
import csv   

def lemmatize(doc, subset):
    '''
    Performs lemmization of input documents.
    Args:
    - docs: one column of a dataframe
    Output:
    - list (nb of contributions) of list (one cell,(size 2: with position input, lemmatized input)) of list of strings
    '''

    
    set_gpu_allocator("pytorch")
    require_gpu()
    set_active_gpu(0)
    if prefer_gpu():
        print("Using GPU!")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        use_pytorch_for_gpu_memory()

        print("GPU Usage")
        GPUtil.showUtilization()

    nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
#   modèle transformer basé sur CamemBERT. Favorise la précision au lieu de l'efficacité :
#     nlp = spacy.load("fr_dep_news_trf", disable = ["parser", "ner"])


    spacy.require_gpu()

    nlp.max_length = 3000000 #this is possible as we're not using parser or ner.

    t0 = time.time()
    
    len_data = doc.shape[0]
    nb_batch = len_data // subset
    size_last_batch = len_data%subset
    first_idx = 0
    last_idx = 0
        
    for batch_nb in range(nb_batch) :
        t1 = time.time()
        lemma_text_list = []
        lemma_pos_list = []
        
        if batch_nb == nb_batch -1 :
            last_idx += size_last_batch
        else :
            last_idx += subset
        
        batch = doc[first_idx : last_idx]
        
        for line in nlp.pipe(batch):
#             ta = time.time()
            lemma_pos_list.append([x.pos_ for x in line])
#             GPUtil.showUtilization()

            lemma_text_list.append([x.lemma_ for x in line])
#             tc = time.time()
#             print(f"spacy computing time per line: {tc-ta}")

        print("GPU Usage")
        GPUtil.showUtilization()
        
        if batch_nb == 0 :
            with open('lemma_temp.csv', 'w', encoding = "utf-8") as f :
                writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_NONE, escapechar = '\\')
                writer.writerow(lemma_text_list)
                f.close()
              
            with open('pos_temp.csv', 'w', encoding = "utf-8") as f :
                writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_NONE, escapechar = '\\')
                writer.writerow(lemma_pos_list)
                f.close()
            
        else :
            with open(r'lemma_temp', 'a', encoding = "utf-8") as f:
                writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_NONE, escapechar = '\\')
                writer.writerow(lemma_text_list)
                f.close()
                
            with open(r'pos_temp', 'a', encoding = "utf-8") as f:
                writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_NONE, escapechar = '\\')
                writer.writerow(lemma_pos_list)
                f.close()
        
        del lemma_pos_list
        del lemma_text_list
#             tokvecs = line._.trf_data.tensors[-1] pour l'utilisation de transformers
#     for doc in nlp.pipe(doc):
#         buffer = []
#         lemma_text_list.append([[" ".join(x.pos_ for x in doc)], [" ".join(x.lemma_ for x in doc)]])     
#        on ne join() pas à ce moment là car la list reste nécessaire pour enlever les stopwords.

        t2 = time.time()
        print(f"end of batch: {batch_nb}, time: {t2-t1}")
        
    t3 = time.time()
    print("Total time: {}".format(t3-t0))

In [11]:
from joblib import Parallel, delayed

def chunker(iterable, total_length, chunksize):
    print("chunker")
    GPUtil.showUtilization()
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=64):
        preproc_pipe.append(lemmatize_pipe(doc))
        print("spacy pipe ")
        GPUtil.showUtilization()

    return preproc_pipe

def preprocess_parallel(texts, chunksize=100):
    executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
    print(1)
    do = delayed(process_chunk)
    print(2)
    tasks = (do(chunk) for chunk in chunker(texts, texts.shape[0], chunksize=chunksize))
    print(3)
    result = executor(tasks)
    return flatten(result)

In [ ]:
%%time
df['preproc_parallel'] = preprocess_parallel(df['clean'], chunksize=1024)

# AttributeError: Can't get attribute 'process_chunk' on <module '__main__' (built-in)>

1
2
chunker
| ID | GPU | MEM |
------------------
|  0 |  3% |  8% |
3


In [13]:
# GPU Total time: 130.2461621761322 (without if and with fr_core_news_md)
lemmatize(df["clean"], 128)

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  7% | 15% |
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 33% |
end of batch: 0, time: 4.81217622756958
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 51% |
end of batch: 1, time: 3.842679023742676
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 52% |
end of batch: 2, time: 4.077171564102173
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  2% | 62% |
end of batch: 3, time: 5.729888916015625
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 62% |
end of batch: 4, time: 5.670403957366943
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 66% |
end of batch: 5, time: 7.172628879547119
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 66% |
end of batch: 6, time: 7.282472372055054
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 66% |
end of batch: 7, time: 8.261136531829834
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% 

end of batch: 74, time: 82.20615196228027
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 85% |
end of batch: 75, time: 84.92238473892212


KeyboardInterrupt: 

In [ ]:
df["lemmatized_text"] = lemmatized_lemma

In [ ]:
df["lemmatized_pos"] = lemmatized_pos

In [ ]:
# pour afficher l'ensemble des cells, à n'utiliser que pour afficher une portion seulement

# pd.set_option('display.max_colwidth', None)

In [ ]:
# df

In [ ]:
df.to_csv("lemmatized_df.csv", index = False)